In [4]:
import numpy as np
import os
import glob
import pickle
import music21 as m21
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, stream, note, chord


# First Create The Note Object From Midi Files

In [35]:
#This function transposes all the songs to either cmaj or amin, so that the model does not have to learn every key
def transpose_song(song):
    
    #get the key for the song
    parts = song.getElementsByClass(m21.stream.Part)
    measure = parts[0].getElementsByClass(m21.stream.Measure)
    key = measure #### This might be working ... ? Can see the key within the music 21 object
    
    
    #estimate the key using m21
    if not isinstance(key, m21.key.Key):
        key = song.analyze("key")
        
    #get the interval for the song to transpose by either major or minor
    if key.mode == "major":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("C"))
    elif key.mode == "minor":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("A"))
        
    transposed_song = song.transpose(interval)
    
    return transposed_song 
        
    

In [37]:
def notes_from_midi():
    #create a note objects off all notes, chords, and rests from midi files
    notes = []
    
    for file in glob.glob('midi_files/*.mid'):
        midi = converter.parse(file)
        #Transpose the songs before we create the note object in the next function
        song = transpose_song(midi)
        #print(midi) -> To make sure it is creating a music stream object
        
        #notes to parse
        notes_to_parse = None
        
        try: #songs have multiple paino parts
            p_parts = instrument.partitionByInstrument(song)
#             print(len(p_parts)) # just checking to see if the output is what I want
#             print(p_parts[1])
            notes_to_parse = p_parts[0].recurse()
        except: #when note is not a chord
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch) + " " + str(element.quarterLength))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder) + str(element.quarterLength))
            elif isinstance(element, note.Rest):
                notes.append(str(element.name) + " " + str(element.quarterLength))
                
#run this when you are ready to save the notes make sure they are correct first.
    with open('data/t_notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
        
    return notes        

In [3]:
def prepare_notes():
    '''
    prepare the notes to be the input and output used by the network
    
    notes = note object created after parsing the midi files using M21
    
    Output: The input and output sequences to the LSTM network
    
    '''
    
    pickle_file = open("data/notes", "rb")
    notes = pickle.load(pickle_file)
    #setting the sequence length to 100
    #print(len(set(notes)))
    sequence = 100 
    
    #creating all the unique notes for create the dictionary from
    pitches = sorted(set(note for note in notes))
    
    #creating the note to int dict to map pitches to integers
    note_dict = dict((note, number) for number, note in enumerate(pitches))
    #print(note_dict)
    lstm_input = []
    lstm_output = []
    
    #creating inputs and corresponding outputs
    for i in range(0, len(notes)- sequence, 1):
        inputs = notes[i : i + sequence]
        outputs = notes[i + sequence]
        lstm_input.append([note_dict[pitch] for pitch in inputs])
        lstm_output.append(note_dict[outputs])
    
    #creating all the objects to reshape network input to make compatable with lstm network
    shape_1 = lstm_input
    shape_2 = len(lstm_input)
    shape_3 = sequence 
    
    #reshaping lstm input for lstm
    lstm_input = np.reshape(shape_1, (shape_2, shape_3, 1))
    
    #normalize lstm input with  number of unique notes
    lstm_input = lstm_input / float(len(pitches))
    #one_hot_encoding lstm_ input
    lstm_output = to_categorical(lstm_output)
    
    return lstm_input, lstm_output

In [5]:
lstm_input, lstm_output = prepare_notes()

In [21]:
print (lstm_input.shape)

(68473, 100, 1)


In [8]:
notes = pickle.load(pickle_file)

NameError: name 'pickle_file' is not defined

In [45]:
pickle_file = open("data/notes", "rb")
notes = pickle.load(pickle_file)

In [13]:
pitches = sorted(set(note for note in notes))

In [16]:
note_dict = dict((note, number) for number, note in enumerate(pitches))

In [38]:
notes_2 =  notes_from_midi()

<music21.stream.Score 0x7fb191824b10>
<music21.stream.Score 0x7fb190c30c10>
<music21.stream.Score 0x7fb192a33750>
<music21.stream.Score 0x7fb18d9ede90>
<music21.stream.Score 0x7fb18e144990>
<music21.stream.Score 0x7fb1ac0e9e10>
<music21.stream.Score 0x7fb18d7e2510>
<music21.stream.Score 0x7fb18d7c7fd0>
<music21.stream.Score 0x7fb18e6ef410>
<music21.stream.Score 0x7fb1917b9d10>
<music21.stream.Score 0x7fb18f1b5b90>
<music21.stream.Score 0x7fb18d952950>
<music21.stream.Score 0x7fb18e0b9710>
<music21.stream.Score 0x7fb1aa7be690>
<music21.stream.Score 0x7fb19177add0>
<music21.stream.Score 0x7fb193928d50>
<music21.stream.Score 0x7fb1ab5ecf10>
<music21.stream.Score 0x7fb18c985210>
<music21.stream.Score 0x7fb18f77e510>
<music21.stream.Score 0x7fb18f442210>
<music21.stream.Score 0x7fb19400e310>
<music21.stream.Score 0x7fb190f78390>
<music21.stream.Score 0x7fb18f0a8d50>
<music21.stream.Score 0x7fb192a261d0>
<music21.stream.Score 0x7fb18d3ad890>
<music21.stream.Score 0x7fb192ee8490>
<music21.str

In [41]:
len(set(notes_2)

2368

In [52]:
pitches = sorted(set(note for note in notes_2))
len(pitches)

2368

In [53]:
notes_2

['rest 1.0',
 'F3 0.5',
 'C4 0.5',
 '4.90.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '2.50.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '9.00.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 'rest 1.0',
 'F3 0.5',
 'C4 0.5',
 '4.90.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '2.50.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '9.00.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 'rest 1.0',
 'F3 0.5',
 'C4 0.5',
 '4.90.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '2.50.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '9.00.5',
 'rest 0.5',
 'E4 0.5',
 'C4 0.5',
 '7.110.5',
 'F3 0.5',
 'C5 0.5',
 'C4 0.5',
 'D5 0.5',
 'rest 0.5',
 '5.9.02.25',
 'C4 0.5',
 'rest 0.5',
 'C4 0.5',
 'rest 0.5',
 'C4 0.5',
 'rest 1.25',
 'F3 0.5',
 'C4 0.5',
 '4.90.25',
 'rest 0.5',
 'F4 0.25',
 'A4 0.25',
 'C4 0.5',
 'E5 0.25',
 '2.50.25',
 'rest 0.5',
 'F4 0.25',
 'A4 0.25',
 'C4 0.5',
 'D5 0.25',
 '9.00.25',
 'rest 0.5',
 'F4 0.25',
 'A4 0.25',
 'C4 0.5',
 'C5 0.25',
 'rest 1.0',
 'F3 0.5',
 'C4 0.5',
 '4.90.25',
 'rest 0.5',
 'F4 0.25',
 'A4 0.25',
 'C

In [51]:
pitch = sorted(set(note for note in notes))
len(pitch)

2424